In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [2]:
import os
import getpass

# Get NVIDIA API key securely (will not display when typing)
nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

# Optional: Verify the key is set (will only show length, not the actual key)
print(f"NVIDIA API key is set with length: {len(os.environ['NVIDIA_API_KEY'])}")

NVIDIA API key is set with length: 70


In [3]:
# Initialize the LLM
llm = ChatNVIDIA(
  model="meta/llama-3.3-70b-instruct"
)

In [4]:
# Define specialized tools for each agent
search_tool = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Web Search",
        func=search_tool.run,
        description="Useful for searching information on the internet"
    )
]

In [5]:
# Simplified prompts with all required variables
RESEARCHER_TEMPLATE = """You are an expert research agent. Your task is to gather comprehensive information.

Available Tools:
{tools}

Use this format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
{agent_scratchpad}"""

ANALYST_TEMPLATE = """You are an expert analysis agent. Analyze the following information and provide insights.

Available Tools:
{tools}

Tool Names:
{tool_names}

Question: {input}
Thought: Let me analyze this information carefully.
{agent_scratchpad}"""

In [6]:
# Create prompts with all required variables
researcher_prompt = PromptTemplate(
    template=RESEARCHER_TEMPLATE,
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"]
)

analyst_prompt = PromptTemplate(
    template=ANALYST_TEMPLATE,
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"]
)

In [7]:
# Initialize agents
researcher = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=researcher_prompt
)

analyst = create_react_agent(
    llm=llm,
    tools=[],  # Empty tools list for analyst
    prompt=analyst_prompt
)

In [8]:
# Create executors
researcher_executor = AgentExecutor(
    agent=researcher,
    tools=tools,
    verbose=True
)

analyst_executor = AgentExecutor(
    agent=analyst,
    tools=[],
    verbose=True
)

In [9]:
def research_and_analyze(query):
    try:
        # Research phase
        research_results = researcher_executor.invoke(
            {
                "input": query,
                "agent_scratchpad": "",
                "tools": tools,
                "tool_names": [tool.name for tool in tools]
            }
        )
        print("\nResearch phase completed.")
        
        # Analysis phase
        analysis = analyst_executor.invoke(
            {
                "input": f"Based on this research: {research_results['output']}, what are the key insights and implications?",
                "agent_scratchpad": "",
                "tools": "No tools available",  # Placeholder for required variable
                "tool_names": "No tools available"  # Placeholder for required variable
            }
        )
        print("\nAnalysis phase completed.")
        
        return {
            "research": research_results['output'],
            "analysis": analysis['output']
        }
    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        print(f"Error details: {type(e).__name__}")
        return {
            "error": str(e),
            "research": None,
            "analysis": None
        }

In [10]:
# Test the workflow directly
query = "What are the latest developments in quantum computing?"
print(f"\nStarting research on: {query}")
results = research_and_analyze(query)
if results.get("error"):
    print(f"\nError occurred: {results['error']}")
else:
    print("\nResearch Results:")
    print(results['research'])
    print("\nAnalysis:")
    print(results['analysis'])


Starting research on: What are the latest developments in quantum computing?


> Entering new AgentExecutor chain...
Thought: To find the latest developments in quantum computing, I need to search for recent news and advancements in this field. The best way to do this is by using a web search.

Action: Web Search
Action Input: Latest developments in quantum computing
MIT physicists predict exotic form of matter with potential for quantum computing. New work suggests the ability to create fractionalized electrons known as non-Abelian anyons without a magnetic field, opening new possibilities for basic research and future applications. November 18, 2024. Read full story → The tech giants say their respective developments are massive advancements in the quantum computing field, which could eventually lead to major social advancements such as discovering new drugs ... Harnessing a new type of material. All of today's announcements build on our team's recent breakthrough: the world's first